In [1]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
#     "unsloth/llama-2-7b-bnb-4bit",
#     "unsloth/gemma-7b-bnb-4bit",
#     "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
#     "unsloth/gemma-2b-bnb-4bit",
#     "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
#     "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import pandas as pd
from datasets import Dataset

# Load the dataset
dataset = pd.read_csv('dataset.csv')
dataset

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
import pandas as pd
from datasets import Dataset

# Load the dataset
dataset = pd.read_csv('Contextdata.csv')

# Verify the dataset structure before conversion
print("Original DataFrame:")
print(dataset.head())
print(dataset.columns)

# Define the data prompt template
data_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response in frensh that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Define the end-of-sequence token
EOS_TOKEN = tokenizer.eos_token

# Define the formatting function
def formatting_prompts_func(examples):
    instructions = examples["question"]
    inputs = examples["context"]
    outputs = examples["answer"]

    texts = []
    for instruction, input_, output in zip(instructions, inputs, outputs):
        # Format the text and add the EOS_TOKEN
        text = data_prompt.format(instruction, input_, output) + EOS_TOKEN
        texts.append(text)
    # Keep the original columns and add the new 'text' column
    examples["text"] = texts
    return examples

# Convert the pandas DataFrame to a Dataset object
dataset = Dataset.from_pandas(dataset)

# Verify the dataset structure before applying the formatting function
print("Original Dataset object:")
print(dataset)

# Apply the formatting function using map
formatted_dataset = dataset.map(formatting_prompts_func, batched=True)

# Verify the formatted data
print("Formatted Dataset object:")
print(formatted_dataset)
print(formatted_dataset.column_names)  # Print the column names to verify


Original DataFrame:
                                            question  \
0  Quand a été créée la Faculté des Sciences et T...   
1  Pouvez-vous me renseigner sur l'historique de ...   
2  Je souhaiterais avoir des informations sur l'a...   
3       Quelle est la date de naissance de la FSTT ?   
4  La Faculté des Sciences et Techniques de Tange...   

                                              answer  \
0  La Faculté des Sciences et Techniques de Tange...   
1  La Faculté des Sciences et Techniques de Tange...   
2  La Faculté des Sciences et Techniques de Tange...   
3  La Faculté des Sciences et Techniques de Tange...   
4  La Faculté des Sciences et Techniques de Tange...   

                                             context  
0  La question porte sur l'année de fondation de ...  
1  La question concerne l'historique de création ...  
2  Cette question demande des détails sur la long...  
3  La question cherche à connaître l'année de fon...  
4  Cette question explore l'aff

Map:   0%|          | 0/1613 [00:00<?, ? examples/s]

Formatted Dataset object:
Dataset({
    features: ['question', 'answer', 'context', 'text'],
    num_rows: 1613
})
['question', 'answer', 'context', 'text']


In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 10,
    # packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 600,
        learning_rate = 8e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=10):   0%|          | 0/1613 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [6]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.605 GB of memory reserved.


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,613 | Num Epochs = 6
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 600
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.321600
2,2.366600
3,2.088300
4,1.919100
5,1.315400
6,1.179900
7,1.069700
8,1.278800
9,1.065600
10,1.027000


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    data_prompt.format(
        "resumer le mot de doyen de la FST tanger", # instruction
        "",# input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    data_prompt.format(
        "Quel sentiment le Greenology Club souhaite-t-il éveiller chez les étudiants concernant l'environnement?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 5000)

In [9]:

# Save the model to Google Drive
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the directory where you want to save the model
output_dir = '/content/drive/My Drive/'

# Make sure the directory exists
os.makedirs(output_dir, exist_ok=True)

# Save the trained model
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model saved to /content/drive/My Drive/


In [60]:
model.save_pretrained("lora_model")